<a href="https://colab.research.google.com/github/daorhe2272/analisis_de_datos_I/blob/main/Taller_4_An%C3%A1lisis_de_Datos_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTAS DEL PROFESOR:** Realiza la preparación completa de los datos de tu dataset seleccionado para el proyecto final. Asegúrate de aplicar las técnicas de escalado, normalización, codificación (One Hot Encoding y Label Encoding), y cualquier otra transformación necesaria para optimizar tus datos para el análisis y modelado.

# **Taller 4: Comparte la preparación de los datos**

Este taller se desarrolla en el contexto de la pregunta SMART definida en el taller 1 y el análisis exploratorio realizado en los talleres 2 y 3.

In [40]:
# Cargamos las librerías necesarias
import pandas as pd
import numpy as np

In [41]:
# Cargamos el dataset
data_url = "https://raw.githubusercontent.com/daorhe2272/analisis_de_datos_I/main/query_results.json"
df = pd.read_json(data_url)

In [42]:
# Revisamos la estructura de los datos
df.head()

,Activo,Categorias,_id,Codigo,Descripcion,Marca,Unidad_venta,Stock,Costo,Precio_base,Rotacion_actual,Rotacion_mes,Rotacion_bimestre,Rotacion_trimestre,Comprado,Fecha_compra,iva,Descripcion_full
0,True,"[Papel regalos, seda, crepé, craft]",63e570ea6aa0ed10c8a8340b,1375,PAPEL CELOFAN AMARILLO 96X100,AFA DIST,UNIDAD,499,314,440,0,530,660,1210,4000,20231204,19,Papel celofan amarillo 96x099
1,False,[Otros cacharrería],63e570ea6aa0ed10c8a833e3,1003,INGRESO PARA TERCEROS (22937),BELLEZA,UNID.,0,1,1380000,0,0,0,0,0,20200210,0,Ingreso para terceros
2,False,[Cuadernos y agendas],63e570ea6aa0ed10c8a833ed,7861084254023,CUAD.LINEA AZUL 85-100-1 STIKER,A&C ESCO,UNID.,0,1450,1800,0,4,4,4,300,20190215,0,Cuaderno Línea Azul con stickers argollado peq...
3,True,[Bombillos],63e570ea6aa0ed10c8a83467,7703255298989,BOMBILLO NALPILUX LED 5W,PANASONI,UNIDAD,130,2285,2800,22,66,166,166,100,20240822,19,Bombillo Nalpilux led 5w
4,False,[Enjuages bucales],63e570ea6aa0ed10c8a83487,037000956662,ENJ.CREST SCOPE OUTLAST X500ML,PREBEL,UNIDAD,0,10042,11520,0,0,0,0,3,20230118,19,Enjuague bucal crest scope outlast de 500 ml


Si observamos la columna categorías, es claro que sus datos contienen listas de conjuntos de cadenas. Necesitamos separar dichas categorías en filas diferentes para poder analizar nuestro conjunto de datos correctamente.

In [43]:
# Usamos el método explode para que cada categoría en su respectiva lista tenga su propia fila
df = df.explode("Categorias")

In [44]:
# Como hemos importado los datos desde un archivo JSON, revisamos la tipología de los datos por columna:
df.dtypes

,0
Activo,bool
Categorias,object
_id,object
Codigo,object
Descripcion,object
Marca,object
Unidad_venta,object
Stock,int64
Costo,int64
Precio_base,int64


Hay varias columnas con tipología de datos object que necesitan ser modificadas y otras más que no necesitamos para nuestro análisis. (Si bien hay algoritmos para automatizar la selección de features, aún no disponemos del conocimiento para emplear estas herramientas por lo que escogemos las columnas de acuerdo al trabajo realizado en los talleres pasados).

In [45]:
# Eliminamos las columnas que no necesitaremos en este análisis:
columns_to_drop = ["Activo","Marca","Costo","Rotacion_actual","Comprado","iva","Stock","Unidad_venta","Codigo"]
df = df.drop(columns=columns_to_drop)

In [46]:
# Reasignamos el tipo de datos donde hace falta:
df = df.astype({"Categorias": "string", "Descripcion": "string", "Fecha_compra": "string", "Descripcion_full": "string" })
df.dtypes

,0
Categorias,string[python]
_id,object
Descripcion,string[python]
Precio_base,int64
Rotacion_mes,int64
Rotacion_bimestre,int64
Rotacion_trimestre,int64
Fecha_compra,string[python]
Descripcion_full,string[python]


In [47]:
# Ahora revisamos los valores nulos en nuestro dataset
print("Valores nulos por columna:\n")
print(df.isnull().sum())

Valores nulos por columna:

Categorias            706
_id                     0
Descripcion             0
Precio_base             0
Rotacion_mes            0
Rotacion_bimestre       0
Rotacion_trimestre      0
Fecha_compra            0
Descripcion_full        0
dtype: int64


In [48]:
df.shape

(11280, 9)

Excepto por la columna Categorias, no hay columnas con valores nulos. No es práctico imputar estos datos ya que sería necesario revisar producto por producto para determinar su categoría. Así mismo, los valores de esta columna son esenciales para nuestro análisis multivariado, por lo que vamos a eliminar las filas con productos sin categorías.

In [49]:
# Eliminamos las filas de productos sin una categoría:
# (estamos asumiendo que los valores faltantes son Missing Completely At Random)
df = df.dropna(subset=["Categorias"])

In [50]:
# ¿Queda alguna fila con valores nulos?
df["Categorias"].isnull().sum() != 0

False

In [51]:
df.shape

(10574, 9)

Tenemos más de 10.000 productos para analizar. Sin embargo, buena parte de estos productos corresponden a códigos en desuso, ya sea por ofertas que ya expiraron, productos que se descontinuaron, o cambios en códigos de barras. Para estos casos, no es necesario ni deseable incluir dichos productos en nuestro análisis.

In [52]:
# **NO CORRER ESTE PASO MÁS DE UNA VEZ**
# Primero removemos los productos comprados hace más de 1 año

# Hay 2 filas con formatos de fecha errados que vamos a remover para poder correr el código

# Los valores de fecha deben tener exactamente 8 caracteres
df["Fecha_compra"] = df["Fecha_compra"].astype(str)
df = df[df["Fecha_compra"].str.len() == 8].copy()

# Removemos valores nulos en caso de haberlos para correr la siguiente línea de código
df = df.dropna()

# Eliminamos las filas con fechas fuera de rango (cuyo año no empieza por 20...)
df = df[df['Fecha_compra'].str.startswith('20')]

from datetime import datetime, timedelta

# Asignamos formatos de fecha para hacer nuestro cálculo
df['Fecha_compra'] = pd.to_datetime(df['Fecha_compra'].astype(str), format='%Y%m%d')

# Fijamos el valor de nuestro filtro a la fecha de hoy menos 365 días
now = datetime.now()
time_limit = now - timedelta(days=365)

# Finalmente eliminamos las filas que cumplen la condición establecida
df = df[df["Fecha_compra"] > time_limit]

df = df.reset_index(drop=True)
df.shape

(5693, 9)

In [53]:
# Ahora eliminamos productos con una rotación trimestral acumulada menor a $100.000
condicion = (df["Rotacion_trimestre"] * df["Precio_base"]) >= 100000
df = df[condicion].copy()
df.shape

(3433, 9)

Continuando con la limpieza y transformación de nuestro dataset, vamos a convertir la rotación trimestral y la rotación bimestral a valores mensuales de ventas (en lugar de valores acumulados).

In [54]:
# **NO CORRER ESTE PASO MÁS DE UNA VEZ**
# Convertimos los valores de rotación acumulada trimestral y bimestral en valores mensuales:
df["Rotacion_trimestre"] = df["Rotacion_trimestre"] - df["Rotacion_bimestre"]
df["Rotacion_bimestre"] = df["Rotacion_bimestre"] - df["Rotacion_mes"]

In [55]:
# Corregimos los nombres de las columnas transformadas:
# Renombramos las columnas de acuerdo al mes del año
from dateutil.relativedelta import relativedelta

mes_1 = (now - relativedelta(months=1)).strftime("%B")
mes_2 = (now - relativedelta(months=2)).strftime("%B")
mes_3 = (now - relativedelta(months=3)).strftime("%B")

df = df.rename(columns={
    "Rotacion_mes": mes_1,
    "Rotacion_bimestre": mes_2,
    "Rotacion_trimestre": mes_3
})

df.columns

Index(['Categorias', '_id', 'Descripcion', 'Precio_base', 'August', 'July',
       'June', 'Fecha_compra', 'Descripcion_full'],
      dtype='object')

# **Transformaciones complementarias**

A continuación vamos a explorar transformaciones adicionales que podrían ser necesarias según la forma definitiva que tome el modelo para el proyecto final. La primera consiste en la normalización de los datos de las ventas para hacer los valores comparables entre uno y otro producto, al menos en términos de su variación mensual.

Queremos comparar la rotación mensual entre diferentes productos, además de hacer estos datos comparables con otros meses, por lo que en lugar de usar una técnica de normalización del tipo Min-Max o Z-Score, vamos a hacer una normalización relativa a un mes específico. En este caso usaremos el mes más antigüo en nuestro dataset (mayo) como referencia.

In [56]:
# Normalizamos los datos de ventas mensuales usando como referencia el mes de mayo:

# Dividir por cero es imposible por lo que debemos imputar los valores donde la rotación de mayo es igual a cero
condicion = df["June"] == 0
imputacion = df.loc[condicion, ['June', 'July', 'August']].mean(axis=1)
imputacion = np.ceil(imputacion).astype(int)

# Reemplazamos los 0 por el promedio mensual de ventas
df.loc[condicion, "May"] = imputacion

# Dividimos cada mes por las ventas de mayo
df["August"] = df["August"] / df["June"]
df["July"] = df["July"] / df["June"]
df["June"] = df["June"] / df["June"]

df.tail()

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Categorias,_id,Descripcion,Precio_base,August,July,June,Fecha_compra,Descripcion_full,May
5642,Shampoo y acondicionadores,66c0aae5984cff0a9c0c9785,SH.TIO NACHO ACLARANTE MANZ.X415ML,32000,inf,NaN,NaN,2024-08-15,Shampoo aclarante Tío Nacho Manzana de 415 ml,7.0
5644,Shampoo y acondicionadores,66c0ab10984cff0a9c0c9e59,AC.MUSS KIDS MANZANI.X 400ML.,18550,inf,NaN,NaN,2024-08-27,Acondicionador Muss Manzanilla de 400 ml,17.0
5659,Colonias y lociones,66cc9e8d7073fa09f3a6885c,LOC.MENTICOL X130 ORIG.+ORIG.X60,10990,inf,NaN,NaN,2024-08-24,LOC.MENTICOL X130 ORIG.+ORIG.X60,6.0
5660,Pañuelos y toallas humectantes,66cc9e8e7073fa09f3a68882,TOA.ARRU.AV.X270 (2X100 Y1X70),24800,inf,NaN,NaN,2024-08-24,TOA.ARRU.AV.X270 (2X100 Y1X70),17.0
5664,Suplementos nutricionales,66cf490c6bc3fa097a1f989d,TARRITO ROJO JGB X1200.5UN.GTS.2DP TRX25,298000,inf,NaN,NaN,2024-08-27,TARRITO ROJO JGB X1200.5UN.GTS.2DP TRX25,1.0


Este método de normalización puede generar valores extremos en casos en los que las ventas del mes de mayo fueron excesivamente bajas debido a una escacez temporal del producto. También pueden haber otros valores extremos que distorcionan los datos de ventas debido a alguna licitación, por ejemplo, con una entidad del gobierno. Podemos corregir parcialmente el problema estableciendo un límite absoluto de 5. Este valor prudencial se estableció consultando con la persona encargada de compras en la organización cuáles valores máximos y mínimos pueden llegar a presentarse en el ciclo de ventas normal. (No tiene sentido usar otros métodos más sofisticados para el manejo de valores extremos cuando solo se tienen datos de ventas para 3 meses).

In [57]:
# Limitamos el valor de las ventas normalizadas a 5
df["July"] = df["July"].clip(lower=-5,upper=5)
df["August"] = df["August"].clip(lower=-5,upper=5)

df.describe()

,Precio_base,August,July,June,Fecha_compra,May
count,3433.000000,3411.000000,3361.000000,3151.0,3433,282.000000
mean,13610.988931,1.932626,1.761815,1.0,2024-06-30 01:30:36.178269696,14.627660
min,50.000000,-5.000000,-5.000000,1.0,2023-09-08 00:00:00,1.000000
25%,3280.000000,0.666667,0.666667,1.0,2024-06-07 00:00:00,3.000000
50%,8700.000000,1.285714,1.200000,1.0,2024-08-01 00:00:00,6.000000
75%,18000.000000,3.000000,2.363636,1.0,2024-08-23 00:00:00,16.000000
max,298000.000000,5.000000,5.000000,1.0,2024-09-05 00:00:00,490.000000
std,18912.230035,1.709657,1.591629,0.0,NaN,33.934514


Nota: Los valores negativos en la rotación no son un error en los datos. Son casos en los que la cantidad devuelta de un producto en un mes superan las ventas del mismo mes. Ejemplo, un producto que se retiró del mercado por problemas de calidad.

Otra manera en la que podemos transformar los datos para el modelo que vamos a emplear en el proyecto final consiste en unificar la rotación mensual de cada producto en una sola columna. De esta manera se generarían varias filas para un mismo producto dependiendo de la cantidad de meses con los que se tienen datos de ventas. En este caso cada mes pasaría a ser una variable para tener en cuenta.

In [58]:
df_mes_vertical = pd.melt(df,
                  id_vars=['Categorias', '_id', 'Descripcion', 'Precio_base', 'Fecha_compra', 'Descripcion_full'],
                  value_vars=['August','July', 'June'],
                  var_name='Month',
                  value_name='Delta_ventas')
df_mes_vertical.head()

,Categorias,_id,Descripcion,Precio_base,Fecha_compra,Descripcion_full,Month,Delta_ventas
0,"Papel regalos, seda, crepé, craft",63e570ea6aa0ed10c8a8340b,PAPEL CELOFAN AMARILLO 96X100,440,2023-12-04,Papel celofan amarillo 96x099,August,0.963636
1,Bombillos,63e570ea6aa0ed10c8a83467,BOMBILLO NALPILUX LED 5W,2800,2024-08-22,Bombillo Nalpilux led 5w,August,5.000000
2,Deshodorantes,63e570eb6aa0ed10c8a834dd,DST.ELIZAB.CREM.X135G.X2 P/ESP.,18500,2024-07-29,Desodorante en crema Elizabeth Arden de 135 gr...,August,1.000000
3,Protección solar,63e570eb6aa0ed10c8a8350d,BLOQ.NUDE SPF 50 X50ML FAC.COLOR,39300,2024-09-03,Bloqueador Nude crema facial con color SPF 50 ...,August,1.428571
4,Protección solar,63e570eb6aa0ed10c8a8351f,BLOQ.NUDE SPF50 X12UN.X10ML.COLOR.FAC.,32400,2024-09-03,BLoqueador Nude crema facial con color SPF 50 ...,August,1.769231


Como podemos apreciar, la (variación de la) rotación ha quedado registrada en la columna Delta_ventas y el mes al que corresponde queda registrado en la columna Month.

# **Conclusiones**

Cada uno de los pasos realizados en este taller para la limpieza y preparación de los datos es un componente fundamental del análisis de datos:

*   La aplanación de los datos en formato JSON nos permite trabajar con los dataframes de pandas.
*   La reasignación de los tipos de datos por columna nos permite aplicar operaciones de concatenamiento o agregación sobre los datos sin vernos afectados por conflictos de compatibilidad de datos.
*   La eliminación de los valores nulos nos permite trabajar con la variable Categorías.
*   La eliminación de productos irrelevantes nos permite construir un modelo más acertado.
*   La normalización de los datos de ventas nor permite realizar comparaciones y asociaciones entre productos o meses de ventas.
*   El tratamiento de los valores extremos reduce la cantidad de ruido en el modelo.
*   La reestructuración de los datos de ventas nos permite analizar los datos desde una nueva perspectiva y construir un modelo con nuevas variables.

Más importante aún es cómo estos pasos se relacionan entre sí para derivar conclusiones verdaderas y pertinentes, basadas en un análisis juicioso de los datos.